In [1]:
!pip install langchain langchain-openai langchain-community faiss-cpu tiktoken langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [2]:
import pandas as pd
import os
from langchain_openai import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cleanerData.csv')
df.rename(columns={'Section': 'Title'}, inplace=True)
df.rename(columns={'Paragraph': 'Content'}, inplace=True)
df['combined'] = df['Title'] + " " + df['Content']
texts = df['combined'].tolist()

In [5]:
loader = TextLoader(texts)

In [6]:
documents = [Document(page_content=t) for t in texts]

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [8]:
docs = text_splitter.split_documents(documents)

In [9]:
len(docs)

4324

In [10]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
library = FAISS.from_documents(docs, embeddings)

In [12]:
library

In [13]:
query1 = "Why can I not work in the UK while applying for asylum?"

In [14]:
queryAnswer = library.similarity_search(query1)

In [15]:
print(queryAnswer[0].page_content)

3, paragraph 1(1)(ga) prevents a local authority in England from providing support or assistance under a provision mentioned in paragraph (ga) to a person if— (a)support is being provided to the person by virtue of paragraph 10B or section 95A of the Immigration and Asylum Act 1999, or (b)there are reasonable grounds for believing that support will be provided to the person by virtue of that paragraph or section.” 7U.K.In paragraph 6 (third class of ineligible person: failed asylum-seeker), in sub-paragraph (1), in the words before sub-paragraph (a), after “person” insert“ in Wales, Scotland or Northern Ireland ”. 8U.K.In paragraph 7 (fourth class of ineligible person: person unlawfully in United Kingdom), in the words before sub-paragraph (a), after “person” insert“ in Wales, Scotland or Northern Ireland ”. 9U.K.Before paragraph 8 insert— 7B(1)Paragraph 1 applies to a person in England if— (a)under the Immigration Act 1971, he requires leave to enter or remain in the United Kingdom


In [16]:
docs_and_scores = library.similarity_search_with_score(query1)

In [17]:
docs_and_scores[0]

(Document(metadata={}, page_content='3, paragraph 1(1)(ga) prevents a local authority in England from providing support or assistance under a provision mentioned in paragraph (ga) to a person if— (a)support is being provided to the person by virtue of paragraph 10B or section 95A of the Immigration and Asylum Act 1999, or (b)there are reasonable grounds for believing that support will be provided to the person by virtue of that paragraph or section.” 7U.K.In paragraph 6 (third class of ineligible person: failed asylum-seeker), in sub-paragraph (1), in the words before sub-paragraph (a), after “person” insert“ in Wales, Scotland or Northern Ireland ”. 8U.K.In paragraph 7 (fourth class of ineligible person: person unlawfully in United Kingdom), in the words before sub-paragraph (a), after “person” insert“ in Wales, Scotland or Northern Ireland ”. 9U.K.Before paragraph 8 insert— 7B(1)Paragraph 1 applies to a person in England if— (a)under the Immigration Act 1971, he requires leave to ent

In [18]:
retriever = library.as_retriever()

In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [22]:
def query_flan(question, model, tokenizer, max_new_tokens=50):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [26]:
def get_answer_from_library(question, library, model, tokenizer, max_new_tokens=50):

    relevant_docs = library.similarity_search(question, k=2)

    print("Relevant Documents:")
    for doc in relevant_docs:
        print(doc.page_content)

    context = " ".join([doc.page_content for doc in relevant_docs])


    prompt = f"Context: {context}\nQuestion: {question}\nAnswer: Provide a detailed answer and explain the reasoning behind it, using the provided context."

    answer = query_flan(prompt, model, tokenizer, max_new_tokens)

    return answer

question = "Why can I not work in the UK while applying for asylum?"
answer = get_answer_from_library(question, library, model, tokenizer)
print("Answer:")
print(answer)

Relevant Documents:
3, paragraph 1(1)(ga) prevents a local authority in England from providing support or assistance under a provision mentioned in paragraph (ga) to a person if— (a)support is being provided to the person by virtue of paragraph 10B or section 95A of the Immigration and Asylum Act 1999, or (b)there are reasonable grounds for believing that support will be provided to the person by virtue of that paragraph or section.” 7U.K.In paragraph 6 (third class of ineligible person: failed asylum-seeker), in sub-paragraph (1), in the words before sub-paragraph (a), after “person” insert“ in Wales, Scotland or Northern Ireland ”. 8U.K.In paragraph 7 (fourth class of ineligible person: person unlawfully in United Kingdom), in the words before sub-paragraph (a), after “person” insert“ in Wales, Scotland or Northern Ireland ”. 9U.K.Before paragraph 8 insert— 7B(1)Paragraph 1 applies to a person in England if— (a)under the Immigration Act 1971, he requires leave to enter or remain in t

In [31]:
import faiss
faiss_index = library.index
faiss.write_index(faiss_index, "library.index")